In [1]:
import os
import sys

In [2]:
current_dir = os.getcwd()
if current_dir.endswith('basic'):
    project_root = os.path.dirname(current_dir)
else:
    project_root = os.path.join(current_dir, '..')
print(project_root)
if project_root not in sys.path:
    print(f"Adding {project_root} to sys.path")
    sys.path.insert(0, project_root)

/home/iwaniukooo/Documents/Projects/wb2-llm/llm-workshop/advanced/..
Adding /home/iwaniukooo/Documents/Projects/wb2-llm/llm-workshop/advanced/.. to sys.path


In [3]:
from basic.llm_model import LLMModel
# from llm_model import LLMModel
from datasets import load_dataset 


/home/iwaniukooo/Documents/Projects/wb2-llm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = load_dataset("go_emotions", "simplified")
with open("dict.txt", "r") as f:
    emotion_dict = [line.strip() for line in f]

emotion_dict

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [5]:


data_subset = dataset['train'].shuffle(seed=43).select(range(4500))  

all_texts = data_subset['text']
all_labels = data_subset['labels'] 
all_labels = [label[0] for label in all_labels]  
all_labels = [emotion_dict[int(label)] for label in all_labels]




In [ ]:
n=25
n_plus=n+5

In [7]:
train_texts, val_texts = all_texts[:n], all_texts[n:n_plus]
train_labels, val_labels = all_labels[:n], all_labels[n:n_plus]

In [8]:
llm = LLMModel()
# llm = LLMModel(model_name="facebook/opt-350m")



Setting up Hugging Face client for model: meta-llama/Llama-3.3-70B-Instruct
Client initialized successfully


In [9]:
llm.train(train_texts,train_labels)

Concept prediction prompt:
In 2 words guess, what task is the model doing, the format is x_test -> y_test:
Yeah for sure, they're too smart to go to class anyway, they've got time. -> admiration
Me too. I have so much shit and I can’t imagine taking any of it into my car and piling it that high. Wtf? -> anger
I get it. However that person doesn’t know who you are... -> neutral
He's both. An edgy neckbeard skinny fat, but he is undeniably funny.  -> amusement
Keep in mind with these verses you've selected you aren't mentioning how we must he crucified in the flesh with [NAME]. -> neutral
What is this task?

Predicted concept: Emotion classification
Training completed with 5 examples


In [10]:
llm.predict(val_texts[0])

Prediction prompt for 'I tried that. It doesn't work out very well. You become a machine.':
You are a classificator
Yeah for sure, they're too smart to go to class anyway, they've got time. -> admiration
Me too. I have so much shit and I can’t imagine taking any of it into my car and piling it that high. Wtf? -> anger
I get it. However that person doesn’t know who you are... -> neutral
He's both. An edgy neckbeard skinny fat, but he is undeniably funny.  -> amusement
Keep in mind with these verses you've selected you aren't mentioning how we must he crucified in the flesh with [NAME]. -> neutral
Learn based on this.

Now predict the output for: I tried that. It doesn't work out very well. You become a machine.
Output (just plain answer):



['neutral']

In [11]:
from custom_model import EmotionClassifier
from custom_model import EmotionBERT
import torch

def main():
    
    classifier = EmotionClassifier(device='cpu')
    
    
    try:
        
        with open("dict.txt", "r") as f:
            emotion_labels = [line.strip() for line in f]
        
        
        classifier._setup_labels(emotion_labels)
        
        
        classifier.model = EmotionBERT(n_classes=len(emotion_labels))
        
        
        classifier.model.load_state_dict(torch.load('best_model_state.pth'))
        
        
        example_text = "i want to cry, depression is coming, life is worth nothing"
        predicted_labels = classifier.predict(example_text)
        
        print(f"Tekst: '{example_text}'")
        print(f"Przewidywane emocje: {predicted_labels}")
        
    except FileNotFoundError:
        print("Błąd: Nie znaleziono pliku modelu 'best_model_state.pth' lub 'dict.txt'")
    except Exception as e:
        print(f"Wystąpił błąd: {str(e)}")

if __name__ == "__main__":
    main()

Labels setup: 28 unique labels found: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']
Tekst: 'i want to cry, depression is coming, life is worth nothing'
Przewidywane emocje: {'disappointment': 0.7149401307106018}
